In [9]:
import pandas as pd
import json
import os
from pathlib import Path

# 全ての行・列を表示
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


def load_all_evaluation_results(base_path):
    """
    Load all evaluation_results.json files from the eval_tree directory structure
    and return a DataFrame with all results.
    """
    results = []
    base_path = Path(base_path)

    # Find all evaluation_results.json files recursively
    for json_file in base_path.rglob("evaluation_results.json"):
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)

            # Extract path information for better identification
            relative_path = json_file.relative_to(base_path)
            path_parts = relative_path.parts[:-1]  # Remove the filename

            # Add path information to the data
            data['full_path'] = str(relative_path.parent)

            # Extract model info from path
            if len(path_parts) > 0:
                data['model_type'] = path_parts[0]
            if len(path_parts) > 1:
                data['model_name'] = path_parts[1] if path_parts[0] != 'fasttext' else 'fasttext'
            if 'template' in data:
                data['template_type'] = data['template']
            elif len(path_parts) > 2 and 'template_' in path_parts[2]:
                data['template_type'] = path_parts[2]

            results.append(data)

        except Exception as e:
            print(f"Error loading {json_file}: {e}")
            continue

    # Convert to DataFrame
    df = pd.DataFrame(results)

    # Sort by model and layer for better readability
    if 'model' in df.columns and 'layer' in df.columns:
        df = df.sort_values(['model', 'layer'])

    return df

# Usage
base_path = "/home/masaki/hierarchical-repr/EntityTree/output/eval_tree"
df = load_all_evaluation_results(base_path)

# Display all results
print(f"Total evaluations loaded: {len(df)}")
print("\nColumns available:", list(df.columns))

# Display summary table
display_columns = ['model', 'layer', 'template', 'jrf_k1', 'jrf_k2']
available_columns = [col for col in display_columns if col in df.columns]

df = df[available_columns]

df = df.rename(columns={
    'jrf_k1': 'jrf_k1(↓)',
    'jrf_k2': 'jrf_k2(↓)'
})


display(df)
layer10_df = df[
    (df['model'] == 'meta-llama/Meta-Llama-3-8B') & 
    (df['layer'] == 10)
]



df = df[df['template'] == 'entity_only']




# # Optional: Save to CSV
# df.to_csv('all_evaluation_results.csv', index=False)
# print(f"\nResults saved to 'all_evaluation_results.csv'")


Total evaluations loaded: 155

Columns available: ['dataset', 'model', 'layer', 'template', 'n_entities', 'jrf_k1', 'jrf_k2', 'gold_internal_nodes', 'pred_internal_nodes', 'full_path', 'model_type', 'model_name', 'template_type']


,model,layer,template,jrf_k1(↓),jrf_k2(↓)
145,fasttext,0,entity_only,90.882977,93.178090
154,gold_binary,0,entity_only,27.730159,30.544391
153,gold_binary_balanced,0,entity_only,82.000000,82.000000
54,gold_binary_left,0,entity_only,82.000000,82.000000
2,gpt2,0,profession_query,90.113621,92.605671
11,gpt2,0,gift,90.113621,92.605671
20,gpt2,0,occupation_question,90.113621,92.605671
29,gpt2,0,professional_intro,90.113621,92.605671
38,gpt2,0,occupation_simple,90.113621,92.605671
47,gpt2,0,entity_only,90.029555,92.570211


In [10]:
layer10_df

,model,layer,template,jrf_k1(↓),jrf_k2(↓)
67,meta-llama/Meta-Llama-3-8B,10,profession_query,88.166667,90.946111
82,meta-llama/Meta-Llama-3-8B,10,gift,88.244444,90.796790
97,meta-llama/Meta-Llama-3-8B,10,occupation_question,88.477778,91.159938
112,meta-llama/Meta-Llama-3-8B,10,professional_intro,88.583333,91.364583
127,meta-llama/Meta-Llama-3-8B,10,occupation_simple,88.560755,91.265806
142,meta-llama/Meta-Llama-3-8B,10,entity_only,86.773810,88.926800


In [11]:
import pandas as pd

# random_embを特定のもの1つだけに絞り込み
# 方法1: インデックスで指定（random_embの中から1つだけ選ぶ）
df_filtered = df[~df['model'].isin(['gold_binary', 'gold_binary_left'])]

# random_embの行を特定（例：最初の1行のみ残す）
random_emb_mask = df_filtered['model'] == 'random_emb'
random_emb_indices = df_filtered[random_emb_mask].index
keep_random_emb_index = random_emb_indices[0]  # 最初の1つだけ残す

# random_embを1つだけ残してフィルタリング
df_filtered = df_filtered[
    (df_filtered['model'] != 'random_emb') | 
    (df_filtered.index == keep_random_emb_index)
]

# または、もしseedやdimなどの列がある場合
# df_filtered = df_filtered[
#     (df_filtered['model'] != 'random_emb') |
#     ((df_filtered['model'] == 'random_emb') & 
#      (df_filtered['seed'] == 42) & 
#      (df_filtered['dim'] == 4096) & 
#      (df_filtered['std'] == 1.0))
# ]

# ソート処理
def extract_model_info(model):
    if model.startswith('random_emb'):
        return ('random_emb', 0)
    elif model.startswith('fasttext'):
        return ('fasttext', 0)
    elif model.startswith('gpt2'):
        return ('gpt2', int(model.replace('gpt2', '') or '0'))
    elif model.startswith('meta-llama'):
        layer_num = model.split('B')[-1]
        return ('meta-llama', int(layer_num or '0'))
    elif model.startswith('gold_binary_balanced'):
        return ('gold_binary_balanced', 0)
    else:
        return ('other', 0)

df_filtered[['model_type', 'layer_num']] = df_filtered['model'].apply(
    lambda x: pd.Series(extract_model_info(x))
)

model_order = {'random_emb': 0, 'fasttext': 1, 'gpt2': 2, 'meta-llama': 3, 'gold_binary_balanced': 4}
df_filtered['model_order'] = df_filtered['model_type'].map(model_order)

df_sorted = df_filtered.sort_values(['model_order', 'layer_num']).drop(['model_type', 'layer_num', 'model_order'], axis=1)

# 表示
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


df_renamed = df_sorted.copy()
df_renamed['model'] = df_renamed['model'].replace({
    'gold_binary_balanced': 'Reference (Gold binary-balanced)',
    'random_emb': 'random_emb (dim=4096)'
})


display(df_renamed)

,model,layer,template,jrf_k1(↓),jrf_k2(↓)
146,random_emb (dim=4096),0,entity_only,91.137045,93.300337
145,fasttext,0,entity_only,90.882977,93.178090
47,gpt2,0,entity_only,90.029555,92.570211
49,gpt2,1,entity_only,90.448459,92.845407
46,gpt2,2,entity_only,90.448276,92.847800
50,gpt2,3,entity_only,90.305556,92.758873
48,gpt2,4,entity_only,89.898407,92.477195
53,gpt2,6,entity_only,89.280844,91.925255
51,gpt2,8,entity_only,88.843137,91.584511
52,gpt2,10,entity_only,89.050000,91.737639
